In [28]:
%cd /content
!pwd
!ls -la

/content
/content
total 677312
drwxr-xr-x 1 root root      4096 Jan 15 03:46 .
drwxr-xr-x 1 root root      4096 Jan 15 02:49 ..
drwxr-xr-x 4 root root      4096 Dec  9 14:41 .config
drwxr-xr-x 3 root root      4096 Jan 15 02:51 data
-rw-r--r-- 1 root root 693539199 Jan 15 03:13 feature_store.zip
drwxr-xr-x 1 root root      4096 Dec  9 14:42 sample_data


In [29]:
%cd /content

REPO_DIR = "/content/repo"
REPO_URL = "https://github.com/malikee00/hm-recommender-system.git"

import os

if not os.path.exists(REPO_DIR):
    !git clone {REPO_URL} {REPO_DIR}
else:
    %cd {REPO_DIR}
    !git pull

%cd {REPO_DIR}
!pwd
!ls -la

/content
Cloning into '/content/repo'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 62 (delta 18), reused 53 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (62/62), 29.88 KiB | 14.94 MiB/s, done.
Resolving deltas: 100% (18/18), done.
/content/repo
/content/repo
total 48
drwxr-xr-x 9 root root 4096 Jan 15 03:53 .
drwxr-xr-x 1 root root 4096 Jan 15 03:53 ..
drwxr-xr-x 2 root root 4096 Jan 15 03:53 data
drwxr-xr-x 2 root root 4096 Jan 15 03:53 features
drwxr-xr-x 8 root root 4096 Jan 15 03:53 .git
-rw-r--r-- 1 root root 1555 Jan 15 03:53 .gitignore
-rw-r--r-- 1 root root  905 Jan 15 03:53 init_project.ps1
drwxr-xr-x 3 root root 4096 Jan 15 03:53 ml
drwxr-xr-x 2 root root 4096 Jan 15 03:53 notebooks
drwxr-xr-x 2 root root 4096 Jan 15 03:53 ops
drwxr-xr-x 3 root root 4096 Jan 15 03:53 pipelines_spark
-rw-r--r-- 1 root root  769 Jan 15 03:53 requirements.txt


In [30]:
!pip -q install -U pip
!pip -q install numpy pandas pyarrow tqdm faiss-cpu gdown torch

import torch
print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())

torch: 2.9.0+cu126
cuda available: True


In [31]:
import os, glob

REPO_DIR = "/content/repo"
CACHE_DIR = "/content/_cache"
ZIP_PATH = os.path.join(CACHE_DIR, "feature_store.zip")
FS_DIR = os.path.join(REPO_DIR, "data", "feature_store")

os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(FS_DIR, exist_ok=True)

parquets = glob.glob(os.path.join(FS_DIR, "*.parquet"))
print("Existing parquet count:", len(parquets))
print("FS_DIR:", FS_DIR)
print("Cache zip exists:", os.path.exists(ZIP_PATH))

Existing parquet count: 0
FS_DIR: /content/repo/data/feature_store
Cache zip exists: False


In [32]:
import os

FILE_ID = "1T-G1_Aa--Pd-xvQBBFO2bpt1XQm8PrRs"  

if not os.path.exists(ZIP_PATH):
    !gdown --id $FILE_ID -O $ZIP_PATH
else:
    print("[DONE] zip already in cache:", ZIP_PATH)

!ls -lh /content/_cache

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1T-G1_Aa--Pd-xvQBBFO2bpt1XQm8PrRs
From (redirected): https://drive.google.com/uc?id=1T-G1_Aa--Pd-xvQBBFO2bpt1XQm8PrRs&confirm=t&uuid=58b44bb4-3e87-4b30-bac1-ba7f3a3e0a86
To: /content/_cache/feature_store.zip
100% 694M/694M [00:08<00:00, 80.8MB/s] 
total 662M
-rw-r--r-- 1 root root 662M Jan 15 03:13 feature_store.zip


In [33]:
import os, glob

REPO_DIR = "/content/repo"
FS_DIR = os.path.join(REPO_DIR, "data", "feature_store")
ZIP_PATH = "/content/_cache/feature_store.zip"

parquets = glob.glob(os.path.join(FS_DIR, "*.parquet"))
if len(parquets) == 0:
    !unzip -o $ZIP_PATH -d $FS_DIR
else:
    print("[DONE] parquet already extracted, skip unzip")

!ls -lh /content/repo/data/feature_store | head -n 20


Archive:  /content/_cache/feature_store.zip
   creating: /content/repo/data/feature_store/feature_store/
   creating: /content/repo/data/feature_store/feature_store/interactions.parquet/
  inflating: /content/repo/data/feature_store/feature_store/interactions.parquet/.part-00000-4ae359bc-c4b2-4257-b9a2-56dc26017ea8-c000.snappy.parquet.crc  
  inflating: /content/repo/data/feature_store/feature_store/interactions.parquet/.part-00001-4ae359bc-c4b2-4257-b9a2-56dc26017ea8-c000.snappy.parquet.crc  
  inflating: /content/repo/data/feature_store/feature_store/interactions.parquet/.part-00002-4ae359bc-c4b2-4257-b9a2-56dc26017ea8-c000.snappy.parquet.crc  
  inflating: /content/repo/data/feature_store/feature_store/interactions.parquet/.part-00003-4ae359bc-c4b2-4257-b9a2-56dc26017ea8-c000.snappy.parquet.crc  
  inflating: /content/repo/data/feature_store/feature_store/interactions.parquet/.part-00004-4ae359bc-c4b2-4257-b9a2-56dc26017ea8-c000.snappy.parquet.crc  
  inflating: /content/repo/data/f

In [35]:
!ls -la /content/repo/data/feature_store
!ls -la /content/repo/data/feature_store/feature_store | head

!cp -r /content/repo/data/feature_store/feature_store/* /content/repo/data/feature_store/

!rm -rf /content/repo/data/feature_store/feature_store

!ls -la /content/repo/data/feature_store | head -n 30


total 12
drwxr-xr-x 3 root root 4096 Jan 15 03:55 .
drwxr-xr-x 3 root root 4096 Jan 15 03:53 ..
drwxrwxrwx 7 root root 4096 Jan 14 15:51 feature_store
total 28
drwxrwxrwx 7 root root 4096 Jan 14 15:51 .
drwxr-xr-x 3 root root 4096 Jan 15 03:55 ..
drwxrwxrwx 2 root root 4096 Jan 14 15:47 interactions.parquet
drwxrwxrwx 2 root root 4096 Jan 14 15:48 item_features.parquet
drwxrwxrwx 2 root root 4096 Jan 14 15:51 item_popularity.parquet
drwxrwxrwx 2 root root 4096 Jan 14 15:48 user_features.parquet
drwxrwxrwx 2 root root 4096 Jan 14 15:50 user_history_agg.parquet
total 28
drwxr-xr-x 7 root root 4096 Jan 15 03:57 .
drwxr-xr-x 3 root root 4096 Jan 15 03:53 ..
drwxr-xr-x 2 root root 4096 Jan 15 03:57 interactions.parquet
drwxr-xr-x 2 root root 4096 Jan 15 03:57 item_features.parquet
drwxr-xr-x 2 root root 4096 Jan 15 03:57 item_popularity.parquet
drwxr-xr-x 2 root root 4096 Jan 15 03:57 user_features.parquet
drwxr-xr-x 2 root root 4096 Jan 15 03:57 user_history_agg.parquet


In [36]:
import pandas as pd

itx = pd.read_parquet("/content/repo/data/feature_store/interactions.parquet")
print(itx.head())
print("rows:", len(itx))
print("cols:", itx.columns.tolist())

                                         customer_id  article_id       t_dat  \
0  aaa78c87aacba903d16f393da3edeca27d62e642b1a639...   706016003  2019-11-29   
1  aaa78c87aacba903d16f393da3edeca27d62e642b1a639...   706016001  2019-11-29   
2  aaa78c87aacba903d16f393da3edeca27d62e642b1a639...   682236013  2019-11-29   
3  aaa78c87aacba903d16f393da3edeca27d62e642b1a639...   706016016  2019-11-29   
4  aaa7a0483dd5b9e395d95324dcbfeb617af9800f39487d...   783335003  2019-11-29   

      price  sales_channel_id  
0  0.025288                 2  
1  0.025288                 2  
2  0.018983                 2  
3  0.025288                 2  
4  0.020322                 2  
rows: 31788324
cols: ['customer_id', 'article_id', 't_dat', 'price', 'sales_channel_id']


In [49]:
%cd /content/repo
!python -m ml.recommender.train --feature_store_dir data/feature_store --registry_dir ml/registry/recommender --embedding_dim 64 --epochs 2 --batch_size 2048 --max_interactions 2000000

/content/repo
[INFO] device: cuda
epoch 1/2: 100% 976/976 [00:07<00:00, 122.22it/s, loss=7.59]
epoch 2/2: 100% 976/976 [00:07<00:00, 131.69it/s, loss=7.59]
item-embeddings: 100% 10/10 [00:00<00:00, 549.58it/s]
/content/repo/ml/recommender/train.py:323: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "trained_at": datetime.utcnow().isoformat() + "Z",
[DONE] Training done. Artifacts saved to: ml/registry/recommender


In [52]:
%cd /content/repo
!git pull
!python -u -m ml.recommender.evaluate --feature_store_dir data/feature_store --registry_dir ml/registry/recommender --reports_dir ml/reports/recommender --ks 5,10,20
!ls -la ml/reports/recommender

/content/repo
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 7 (delta 4), reused 7 (delta 4), pack-reused 0 (from 0)
Unpacking objects: 100% (7/7), 3.98 KiB | 582.00 KiB/s, done.
From https://github.com/malikee00/hm-recommender-system
   7b4797d..d39244e  main       -> origin/main
Updating 7b4797d..d39244e
Fast-forward
 ml/recommender/evaluate.py       |  68 ++---
 notebooks/run_phase3_colab.ipynb | 524 ++++++++++++++++++++++++++++-----------
 2 files changed, 395 insertions(+), 197 deletions(-)
^C
total 8
drwxr-xr-x 2 root root 4096 Jan 15 04:01 .
drwxr-xr-x 3 root root 4096 Jan 15 04:01 ..


In [53]:
%cd /content/repo
!set -x; python -u -m ml.recommender.evaluate --feature_store_dir data/feature_store --registry_dir ml/registry/recommender --reports_dir ml/reports/recommender --ks 5,10,20; echo "EXIT_CODE=$?"

: 

In [39]:
import json

with open("ml/registry/recommender/metadata.json") as f:
    meta = json.load(f)

meta

{'model_type': 'two-tower',
 'embedding_dim': 64,
 'epochs': 2,
 'batch_size': 2048,
 'lr': 0.001,
 'train_rows': 2000000,
 'num_users': 684538,
 'num_items': 81338,
 'trained_at': '2026-01-15T04:01:34.407065Z',
 'device': 'cuda',
 'feature_store_dir': 'data/feature_store',
 'artifacts': {'model': 'two_tower_model.pt',
  'item_embeddings': 'item_embeddings.npy',
  'faiss_index': 'faiss.index',
  'user_id_mapping': 'user_id_mapping.json',
  'item_id_mapping': 'item_id_mapping.json',
  'feature_encoders': 'feature_encoders.json',
  'item_popularity': 'item_popularity.csv'}}

In [40]:
import json

with open("ml/reports/recommender/metrics.json") as f:
    metrics = json.load(f)

with open("ml/reports/recommender/baseline_metrics.json") as f:
    baseline = json.load(f)

print("MODEL:", metrics)
print("BASELINE:", baseline)

FileNotFoundError: [Errno 2] No such file or directory: 'ml/reports/recommender/metrics.json'

In [ ]:
%cd /content/repo
!rm -f recommender_artifacts.zip
!zip -r recommender_artifacts.zip ml/registry/recommender ml/reports/recommender
!ls -lh recommender_artifacts.zip

from google.colab import files
files.download("recommender_artifacts.zip")

In [12]:
!unzip -o /content/feature_store.zip -d data/feature_store

Archive:  /content/feature_store.zip
   creating: data/feature_store/item_popularity.parquet/
  inflating: data/feature_store/item_popularity.parquet/.part-00000-527b7ce3-00ab-4c30-91ab-86306f93169a-c000.snappy.parquet.crc  
  inflating: data/feature_store/item_popularity.parquet/._SUCCESS.crc  
  inflating: data/feature_store/item_popularity.parquet/part-00000-527b7ce3-00ab-4c30-91ab-86306f93169a-c000.snappy.parquet  
  inflating: data/feature_store/item_popularity.parquet/_SUCCESS  
   creating: data/feature_store/user_features.parquet/
  inflating: data/feature_store/user_features.parquet/.part-00000-95e03489-490d-4f0d-bae1-eaa4214d2e0b-c000.snappy.parquet.crc  
  inflating: data/feature_store/user_features.parquet/.part-00001-95e03489-490d-4f0d-bae1-eaa4214d2e0b-c000.snappy.parquet.crc  
  inflating: data/feature_store/user_features.parquet/.part-00002-95e03489-490d-4f0d-bae1-eaa4214d2e0b-c000.snappy.parquet.crc  
  inflating: data/feature_store/user_features.parquet/.part-00003-95

In [ ]:
!ls -lh data/feature_store

In [9]:
!python -m ml.recommender.train \
  --feature_store_dir data/feature_store \
  --registry_dir ml/registry/recommender \
  --embedding_dim 64 \
  --epochs 2 \
  --batch_size 2048 \
  --max_interactions 2000000

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/content/repo/ml/recommender/train.py", line 291, in <module>
    main()
  File "/content/repo/ml/recommender/train.py", line 140, in main
    interactions, user_df, item_df, item_popularity = load_training_tables(args.feature_store_dir)
                                                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/repo/ml/recommender/train.py", line 32, in load_training_tables
    interactions = pd.read_parquet(os.path.join(feature_store_dir, "interactions.parquet"))
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parquet.py", line 667, in read_parquet
    return impl.read(
           ^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parquet.py", line 267, in read
    path_or_handle, 

In [ ]:
!python -m ml.recommender.evaluate \
  --feature_store_dir data/feature_store \
  --registry_dir ml/registry/recommender \
  --reports_dir ml/reports/recommender \
  --ks 5,10,20


In [ ]:
!rm -f recommender_artifacts.zip
!zip -r recommender_artifacts.zip ml/registry/recommender ml/reports/recommender
!ls -lh recommender_artifacts.zip

from google.colab import files
files.download("recommender_artifacts.zip")
